# Setup

In [1]:
# Check out the robocon2017 branch of some of our packages:
Pkg.checkout("RigidBodyDynamics", "robocon2017")

if !isdir(Pkg.dir("DrakeVisualizer"))
    Pkg.clone("https://github.com/rdeits/DrakeVisualizer.jl")
    Pkg.checkout("DrakeVisualizer", "robocon2017")
    Pkg.build("DrakeVisualizer")
end

Pkg.add("RigidBodyTreeInspector")
Pkg.checkout("RigidBodyTreeInspector", "robocon2017")

INFO: Checking out RigidBodyDynamics robocon2017...
INFO: Pulling RigidBodyDynamics latest robocon2017...
INFO: No packages to install, update or remove
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of RigidBodyTreeInspector
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Checking out RigidBodyTreeInspector robocon2017...
INFO: Pulling RigidBodyTreeInspector latest robocon2017...
INFO: No packages to install, update or remove


In [2]:
using RigidBodyDynamics
using StaticArrays
using Plots
using RigidBodyTreeInspector
using DrakeVisualizer

INFO: Recompiling stale cache file /Users/twan/code/julia/robocon2017/lib/v0.5/RigidBodyDynamics.ji for module RigidBodyDynamics.


# Creating a double pendulum `Mechanism`

We're going to create a simple `Mechanism` that represents a [double pendulum](https://en.wikipedia.org/wiki/Double_pendulum). The `Mechanism` type can be thought of as an interconnection of rigid bodies and joints.

One way of creating a `Mechanism` is by parsing a [URDF](http://wiki.ros.org/urdf) file:

In [3]:
urdf = "data/doublependulum.urdf"
doublependulum = parse_urdf(Float64, urdf)

Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: upper_link, Edge: shoulder
      Vertex: lower_link, Edge: elbow

Here are the bodies of our double pendulum:

In [4]:
collect(bodies(doublependulum))

4-element Array{RigidBodyDynamics.RigidBody{Float64},1}:
 RigidBody: "world"     
 RigidBody: "base_link" 
 RigidBody: "upper_link"
 RigidBody: "lower_link"

And here are its joints:

In [5]:
collect(joints(doublependulum))

3-element Array{RigidBodyDynamics.Joint{Float64},1}:
 Joint "base_link_to_world": Fixed joint                 
 Joint "shoulder": Revolute joint with axis [0.0,1.0,0.0]
 Joint "elbow": Revolute joint with axis [0.0,1.0,0.0]   

# The state of a `Mechanism`

A `Mechanism` stores the joint/rigid body layout, but no state information. State information is separated out into a `MechanismState` object:

In [6]:
const state = MechanismState(Float64, doublependulum)

MechanismState{Float64, Float64, Float64}(…)

Let's first set the configurations and velocities of the joints:

In [7]:
fixedjoint, shoulder, elbow = (joints(doublependulum)...)
configuration(state, shoulder)[:] = 0.3
configuration(state, elbow)[:] = 0.4
velocity(state, shoulder)[:] = 1.
velocity(state, elbow)[:] = 2.;

**Important**: a `MechanismState` contains cache variables that depend on the configurations and velocities of the joints. These need to be invalidated when the configurations and velocities are changed. To do this, call

In [8]:
setdirty!(state)

The joint configurations and velocities are stored as `Vector`s (denoted $q$ and $v$ respectively in this package) inside the `MechanismState` object:

In [9]:
@show q = configuration_vector(state)
@show v = velocity_vector(state);

q = configuration_vector(state) = [0.3,0.4]
v = velocity_vector(state) = [1.0,2.0]


# Kinematics

We are now ready to do kinematics. Here's how you transform a point at the origin of the frame after the elbow joint to world frame:

In [10]:
world = root_body(doublependulum)
transform(state, Point3D(elbow.frameAfter, zeros(SVector{3})), default_frame(world))

Point3D in "world": [-0.29552,0.25,-0.955336]

Other objects like `Wrench`es, `Twist`s, and `SpatialInertia`s, etc. can be transformed in similar fashion.

You can also ask for the homogeneous transform to world:

In [11]:
transform_to_root(state, elbow.frameAfter)

Transform3D from "after_elbow" to "world":
rotation: 0.7000000000000001 rad about [0.0,1.0,0.0], translation: [-0.29552,0.25,-0.955336]


Or a relative transform:

In [12]:
relative_transform(state, elbow.frameAfter, shoulder.frameAfter)

Transform3D from "after_elbow" to "after_shoulder":
rotation: 0.40000000000000024 rad about [2.65182e-8,1.0,0.0], translation: [0.0,0.1,-1.0]


Here's the center of mass of the double pendulum:

In [13]:
center_of_mass(state)

Point3D in "world": [-0.543749,0.2,-1.09892]

# Dynamics

A `MechanismState` can also be used to compute quantities related to the dynamics of the `Mechanism`. Here we compute the mass matrix:

In [14]:
mass_matrix(state)

2×2 Symmetric{Float64,Array{Float64,2}}:
 4.50512  2.25106
 2.25106  1.33   

Here's the documentation for `mass_matrix` by the way:

In [15]:
?mass_matrix

search: mass_matrix mass_matrix!



```julia
mass_matrix(state)

```

Compute the joint-space mass matrix (also known as the inertia matrix) of the `Mechanism` in the given state, i.e., the matrix $M(q)$ in the unconstrained joint-space equations of motion

$$
M(q) \dot{v} + c(q, v, w_\text{ext}) = \tau
$$

This method implements the composite rigid body algorithm.


Note that there is also a zero-allocation version, `mass_matrix!` (the `!` at the end of a method is a Julia convention signifying that the function is 'in-place', i.e. modifies its input data).

We can do inverse dynamics as follows (note again that there is a non-allocating version of this method as well):

In [16]:
v̇ = [2.; 3.] # the joint acceleration vector, i.e., the time derivative of the joint velocity vector v
inverse_dynamics(state, v̇)

2-element Array{Float64,1}:
 23.3164
 15.2013

# Simulation

Let's simulate the double pendulum for 5 seconds, starting from the state we defined earlier. For this, we can use the basic `simulate` function:

In [17]:
ts, qs, vs = simulate(state, 5., Δt = 1e-3);

`simulate` returns a vector of times (`ts`) and associated joint configurations (`qs`) and velocities (`vs`).

*Note:* A lower level interface for simulation/ODE integration with more options is also available. Consult the documentation for more information.

We can plot the trajectories using e.g. Plots.jl:

In [18]:
plot(ts, collect(q[1] for q in qs), xlabel = "Time [s]", ylabel = "Angle [rad]", lab = "Shoulder")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 Time [s] 
 
 
 Angle [rad] 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,153.154 54.8786,152.987 54.9869,152.818 55.0951,152.649 55.2034,152.479 55.3116,152.309 55.4199,152.137 55.5282,151.964 55.6364,151.79 55.7447,151.616 
 55.8529,151.441 55.9612,151.264 56.0694,151.087 56.1777,150.909 56.286,150.73 56.3942,150.551 56.5025,150.37 56.6107,150.188 56.719,150.006 56.8273,149.823 
 56.9355,149.639 57.0438,149.454 57.152,149.268 57.2603,149.081 57.3685,148.894 57.4768,148.705 57.5851,148.516 57.6933,148.326 57.8016,148.135 57.9098,147.944 
 58.0181,147.751 58.1264,147.558 58.2346,147.364 58.3429,147.169 58.4511,146.973 58.5594,146.776 58.6676,146.579 58.7759,146.381 58.8842,146.182 58.9924,145.982 
 59.1007,145.782 59.2089,145.58 59.3172,145.378 59.4255,145.176 59.5337,144.972 59.642,144.768 59.7502,144.563 59.8585,144.357 59.9668,144.15 60.075,143.943 
 60.1833,143.735 60.2915,143.526 60.3998,143.317 60.508,143.107 60.6163,142.896 60.7246,142.684 60.8328,142.472 60.9411,142.259 61.0493,142.045 61.1576,141.831 
 61.2659,141.616 61.3741,141.4 61.4824,141.184 61.5906,140.967 61.6989,140.749 61.8071,140.531 61.9154,140.312 62.0237,140.092 62.1319,139.872 62.2402,139.651 
 62.3484,139.429 62.4567,139.207 62.565,138.984 62.6732,138.761 62.7815,138.537 62.8897,138.312 62.998,138.087 63.1062,137.861 63.2145,137.635 63.3228,137.408 
 63.431,137.181 63.5393,136.953 63.6475,136.724 63.7558,136.495 63.8641,136.265 63.9723,136.035 64.0806,135.804 64.1888,135.573 64.2971,135.341 64.4054,135.109 
 64.5136,134.876 64.6219,134.643 64.7301,134.409 64.8384,134.174 64.9466,133.939 65.0549,133.704 65.1632,133.468 65.2714,133.232 65.3797,132.995 65.4879,132.758 
 65.5962,132.52 65.7045,132.282 65.8127,132.044 65.921,131.805 66.0292,131.565 66.1375,131.326 66.2457,131.085 66.354,130.845 66.4623,130.604 66.5705,130.362 
 66.6788,130.12 66.787,129.878 66.8953,129.635 67.0036,129.392 67.1118,129.149 67.2201,128.905 67.3283,128.661 67.4366,128.417 67.5448,128.172 67.6531,127.927 
 67.7614,127.681 67.8696,127.435 67.9779,127.189 68.0861,126.943 68.1944,126.696 68.3027,126.449 68.4109,126.202 68.5192,125.954 68.6274,125.706 68.7357,125.458 
 68.844,125.209 68.9522,124.96 69.0605,124.711 69.1687,124.462 69.277,124.212 69.3852,123.963 69.4935,123.713 69.6018,123.462 69.71,123.212 69.8183,122.961 
 69.9265,122.71 70.0348,122.459 70.1431,122.208 70.2513,121.956 70.3596,121.704 70.4678,121.452 70.5761,121.2 70.6843,120.948 70.7926,120.696 70.9009,120.443 
 71.0091,120.19 71.1174,119.937 71.2256,119.684 71.3339,119.431 71.4422,119.177 71.5504,118.924 71.6587,118.67 71.7669,118.417 71.8752,118.163 71.9834,117.909 
 72.0917,117.655 72.2,117.401 72.3082,117.146 72.4165,116.892 72.5247,116.638 72.633,116.383 72.7413,116.129 72.8495,115.874 72.9578,115.62 73.066,115.365 
 73.1743,115.11 73.2825,114.855 73.3908,114.601 73.4991,114.346 73.6073,114.091 73.7156,113.836 73.8238,113.581 73.9321,113.326 74.0404,113.071 74.1486,112.817 
 74.2569,112.562 74.3651,112.307 74.4734,112.052 74.5817,111.797 74.6899,111.543 74.7982,111.288 74.9064,111.033 75.0147,110.779 75.1229,110.524 75.2312,110.27 
 75.3395,110.015 75.4477,109.761 75.556,109.507 75.6642,109.253 75.7725,108.999 75.8808,108.745 75.989,108.491 76.0973,108.237 76.2055,107.984 76.3138,107.73 
 76.422,107.477 76.5303,107.223 76.6386,106.97 76.7468,106.717 76.8551,106.465 76.9633,106.212 77.0716,105.959 77.1799,105.707 77.2881,105.455 77.3964,105.203 
 77.5046,104.951 77.6129,104.699 77.7211,104.448 77.8294,104.197 77.9377,103.946 78.0459,103.695 78.1542,103.444 78.2624,103.194 78.3707,102.944 78.479,102.694 
 78.5872,102.444 78.6955,102.194 78.8037,101.945 78.912,101.696 79.0203,101.447 79.1285,101.199 79.2368,100.951 79.345,100.703

We can also show a 3D visualization of the double pendulum using DrakeVisualizer.

First, launch the viewer:

In [19]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();

In [20]:
vis = Visualizer(parse_urdf(urdf, doublependulum));

In [21]:
RigidBodyTreeInspector.animate(vis, doublependulum, ts, qs)

# Symbolic dynamics

## Setup

In [22]:
# Make SymPy use version of Python downloaded through Julia's package system instead of system Python
ENV["PYTHON"]=""
Pkg.build("SymPy")

INFO: Building Conda
INFO: Building PyCall
INFO: Using the Python distribution in the Conda package by default.
To use a different Python version, set ENV["PYTHON"]="pythoncommand" and re-run Pkg.build("PyCall").


Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr:
#
numpy                     1.12.0                   py27_0  


INFO: PyCall is using /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr/bin/python (Python 2.7.12) at /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr/bin/python, libpython = /Users/twan/code/julia/robocon2017/v0.5/Conda/deps/usr/lib/libpython2.7
INFO: /Users/twan/code/julia/robocon2017/v0.5/PyCall/deps/deps.jl has not changed
INFO: /Users/twan/code/julia/robocon2017/v0.5/PyCall/deps/PYTHON has not changed


In [23]:
using SymPy

In [24]:
# add a method to Base.flipsign and eps for SymPy. TODO: SymPy PR
Base.flipsign(sym::Sym, sign::Number) = sign > 0 ? sym : -sym;
Base.eps(::Type{SymPy.Sym}) = zero(Sym)

In [25]:
doublependulumsym = parse_urdf(Sym, urdf)
remove_fixed_joints!(doublependulumsym)

Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow

## Create symbolic parameters
* Masses: $m_1, m_2$
* Mass moments of inertia (about center of mass): $I_1, I_2$
* Link lengths: $l_1, l_2$
* Center of mass locations (w.r.t. preceding joint axis): $c_1, c_2$
* Gravitational acceleration: $g$

In [26]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitationalAcceleration = @syms g real = true
params = [inertias..., lengths..., gravitationalAcceleration...]

9-element Array{SymPy.Sym,1}
[m_1]
[   ]
[m_2]
[   ]
[I_1]
[   ]
[I_2]
[   ]
[l_1]
[   ]
[l_2]
[   ]
[c_1]
[   ]
[c_2]
[   ]
[ g ]

Unpack bodies and joints:

In [27]:
world, upperlink, lowerlink = (bodies(doublependulumsym)...)
shoulder, elbow = (joints(doublependulumsym)...);

Set symbolic parameters:

In [28]:
axis = shoulder.jointType.axis
spatial_inertia!(upperlink, SpatialInertia(shoulder.frameAfter, I_1 * axis * axis', SVector(0, 0, -c_1), m_1))
spatial_inertia!(lowerlink, SpatialInertia(elbow.frameAfter, I_2 * axis * axis', SVector(0, 0, -c_2), m_2))
add_frame!(upperlink, Transform3D(elbow.frameBefore, shoulder.frameAfter, SVector(0, 0, l_1)));

Create a symbolic `MechanismState`:

In [29]:
statesym = MechanismState(Sym, doublependulumsym);

Set the joint configuration and joint velocity vectors of the `MechanismState` to new vectors of symbolic variables:

In [30]:
configuration_vector(statesym)[:] = [symbols("q_$i", real = true) for i = 1 : num_positions(statesym)]

2-element Array{SymPy.Sym,1}
[q_1]
[   ]
[q_2]

In [31]:
velocity_vector(statesym)[:] = [symbols("v_$i", real = true) for i = 1 : num_positions(statesym)]

2-element Array{SymPy.Sym,1}
[v_1]
[   ]
[v_2]

Compute the mass matrix in symbolic form:

In [32]:
M = mass_matrix(statesym)
map!(simplify, M.data) # Note: M is a Symmetric matrix type; need to simplify the underlying data
full(M) # convert to full form so that it is pretty-printed (minor bug in SymPy.jl)

2×2 Array{SymPy.Sym,2}
[                                    2                            ]
[I_1 + I_2 - 2*c_2*l_1*cos(q_2) + l_1 *m_2  I_2 - c_2*l_1*cos(q_2)]
[                                                                 ]
[         I_2 - c_2*l_1*cos(q_2)                     I_2          ]

Kinetic energy:

In [33]:
simplify(kinetic_energy(statesym))

       2          2                        2                                  
I_1*v_1    I_2*v_1                  I_2*v_2               2                   
-------- + -------- + I_2*v_1*v_2 + -------- - c_2*l_1*v_1 *cos(q_2) - c_2*l_1
   2          2                        2                                      

                       2        2
                    l_1 *m_2*v_1 
*v_1*v_2*cos(q_2) + -------------
                          2      